In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nanonispy as nap
%matplotlib inline

In [25]:
import scipy.stats as stats

In [32]:
sns.set_style('darkgrid')
plt.style.use('seaborn-darkgrid')

In [2]:
grid_list = []
for i in range(28,32):
    grid_temp = nap.read.Grid('.\STM-1 Run #84 NbSe2 film\Grid Spectroscopy0'+str(i)+'.3ds')
    grid_list.append(grid_temp)
    
grid_temp = nap.read.Grid('.\STM-1 Run #84 NbSe2 film\Grid Spectroscopy035.3ds')
grid_list.append(grid_temp)
grid_temp = nap.read.Grid('.\STM-1 Run #84 NbSe2 film\Grid Spectroscopy037.3ds')
grid_list.append(grid_temp)
grid_temp = nap.read.Grid('.\STM-1 Run #84 NbSe2 film\Grid Spectroscopy039.3ds')
grid_list.append(grid_temp)

In [3]:
bias_vals=grid_list[0].signals.get("sweep_signal")
bias = bias_vals *1000

In [10]:
lockin_flat_list = []
for i in range(7):
    lockin_temp = grid_list[i].signals.get('Lockin X (V)')
    lockin_reshaped = lockin_temp.reshape(30,55,51)
    lower_lockin = lockin_reshaped[:14,:,:]
    lockin_flat_list.append(lower_lockin.reshape(770,51))

In [11]:
ZBC_flat_list = []
for i in range(7):
    if i == 4:
        ZBC_flat = lockin_flat_list[i][:,23]
        ZBC_flat_list.append(ZBC_flat)
    else:
        ZBC_flat = lockin_flat_list[i][:,25]
        ZBC_flat_list.append(ZBC_flat)

In [38]:
rbound = 10
lbound = 51- rbound
interval = (rbound-1)*0.06

lockin_sub_norm = []
for j in range (7):
    lockin_sub_norm.append(np.zeros((770,51)))
    for i in range(770):
        right = (lockin_flat_list[j][i,:rbound].sum())*0.06
        left = (lockin_flat_list[j][i,lbound:51].sum()) *0.06
        slope = (right - left)/(interval*(3-interval))
        Sub_spec = lockin_flat_list[j][i,:]-bias*slope
        if j == 4:
            Norm_spec = Sub_spec/Sub_spec[48]
        else:
            Norm_spec = Sub_spec/Sub_spec[50]
        lockin_sub_norm[j][i] = Norm_spec

In [42]:
ZBC_list = []

for i in range(7):
    if i != 4:
        ZBC_list.append(lockin_sub_norm[i][:,25])
    else:
        ZBC_list.append(lockin_sub_norm[i][:,23])

In [43]:
ZBC_final_list = ZBC_list.copy()
for i in range(7):
    for cyc in range(10):
        Mean = ZBC_final_list[i].mean()
        STD = ZBC_final_list[i].std()
        
        ZBC_final_list[i] = ZBC_final_list[i][(ZBC_final_list[i] > (Mean -5*STD)) & \
                                              (ZBC_final_list[i] < (Mean +5*STD))]

In [35]:
plt.figure()
#sns.distplot(lock_0T_sub_norm[:,25],bins =100,kde=True,fit=stats.norm)
sns.distplot(ZBC_0T,bins =30,kde=True,fit=stats.norm)
plt.xlabel('Zero Bias Conductance',size = 15)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.title('Histogram of Zero Bias Conductance at 0 T',size =20);
plt.savefig('.\STM-1 Run #84 NbSe2 film\Hist of ZBC at 0 T_after line subtraction and normalization.png',dpi=500)
plt.close()

In [45]:
for i in range(7):
    print(ZBC_final_list[i].shape)
    print('\n')

(766,)


(770,)


(769,)


(769,)


(769,)


(770,)


(767,)




In [61]:
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15)

plt.figure(figsize=(16,16))
binslist = [5, 10, 22, 20, 22, 20, 40]
for i in range(1,8): 
    plt.subplot(4,2,i)
    sns.distplot(ZBC_final_list[(i-1)],bins = binslist[(i-1)],kde=True,fit=stats.norm)
    plt.xlabel('Zero Bias Conductance',size = 20)
    plt.xlim((0,1.5))
    plt.ylim((0,35))
    
    if i < 5:
        plt.title('Histogram of Zero Bias Conductance at ' + str((i-1)*0.5) +' T',size =20)
    else:
        plt.title('Histogram of Zero Bias Conductance at ' + str(i-3) +' T',size =20)
    plt.tight_layout()
plt.savefig('.\STM-1 Run #84 NbSe2 film\Hist of ZBC in fields_after line sub and norm to -1.5 meV_same x and y range.png',dpi=500)
plt.close()

In [50]:
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15)

plt.figure(figsize=(16,16))

for i in range(1,8): 
    plt.subplot(4,2,i)
    for j in range(770):
        plt.plot(bias,lockin_sub_norm[(i-1)][j,:],lw=0.5)
    
    plt.xlabel('Bias (mV)',size = 20)
    plt.xlim((0,1.2))
    if i == 1:
        plt.ylim(top=2)
    if i < 5:
        plt.title('Variance at ' + str((i-1)*0.5) +' T',size =20)
    else:
        plt.title('Variance at ' + str(i-3) +' T',size =20)
    plt.tight_layout()
#plt.savefig('.\STM-1 Run #84 NbSe2 film\Variance in fields_after line sub and norm to -1.5 meV.png',dpi=500)
#plt.close()